# Getting started with this notebook

1. Install the nodejs kernel for ipython notebook from here: https://github.com/notablemind/jupyter-nodejs

1. In the folder that this notebook is saved, run the following commands:
```
npm install jquery
npm install jsdom@3.1.2
```

1. Open this notebook and set the kernel to NodeJS (Kernel -> Change Kernel -> NodeJS)

In [11]:
// The "%%coffee" magic lets us use coffeescript in a cell. The downside is that any variables
//     defined in the cell can't be used in other cells. 
//  This is not a problem when writing in javascript.
%load_ext coffee

In [12]:
// Little bit of finagling to trick jquery to work inside Jupyter without a real window.document
$ = null
env = require('jsdom').env;
html = '<html></html>'

env(html, function (errors, window) {
    $ = require('jquery')(window);
  });

# Asynchronous != Multithreaded
Javascript is single threaded: that's why we can't wait for responses to come back, like in Python. If we wait, nothing else can happen. The website appears frozen because even UI events can't be processed. This is also why writing fast code is so important with javascript: if your code is running, the UI is frozen.

In [13]:
%%coffee
blocking_request = ->
  return $.ajax(
    async: false  # Mimic python requests
    url: 'http://www.httpbin.org/delay/1'
    type: 'GET'
  )
    
console.log("Starting blocking request...\n")
console.log(blocking_request())
console.log('\nRequest complete! UI unblocked.')

Starting blocking request...

{"readyState":4,"responseText":"{\n  \"args\": {}, \n  \"data\": \"\", \n  \"files\": {}, \n  \"form\": {}, \n  \"headers\": {\n    \"Accept\": \"*/*, */*\", \n    \"Host\": \"www.httpbin.org\", \n    \"User-Agent\": \"node-XMLHttpRequest\"\n  }, \n  \"origin\": \"209.117.82.227\", \n  \"url\": \"http://www.httpbin.org/delay/1\"\n}\n","responseJSON":{"args":{},"data":"","files":{},"form":{},"headers":{"Accept":"*/*, */*","Host":"www.httpbin.org","User-Agent":"node-XMLHttpRequest"},"origin":"209.117.82.227","url":"http://www.httpbin.org/delay/1"},"status":200,"statusText":"success"}

Request complete! UI unblocked.


# Callbacks
If we can't do work on another thread, how do we make sure we can do work after the asynchronous call finishes? We can pass in a callback function that will get called once the request comes back. This lets the request function return immediately and not block.

In [15]:
%%coffee
make_request = (callback) ->
  $.ajax(
    url: 'http://www.httpbin.org/delay/2'
    type: 'GET'
    success: (response) ->
      callback(response)
  )

console.log("Starting request...")

make_request((response) ->
  console.log("Request complete!")
)

console.log('Request made! UI unblocked.\n')

Starting request...
Request made! UI unblocked.

Request complete!


## Drawbacks: You only get One
You only get one function to do everything, which means that if you need to do more requests, these requests have to live inside the callback. Each request adds another layer to the callback, nesting the code and making it hard to read and reason about. This is sometimes referred to as the "pyramid of doom" or "callback hell".

In [ ]:
%%coffee
make_requests = (callback) ->
  $.ajax(
    url: 'http://www.httpbin.org/delay/1'
    type: 'GET'
    success: (response) ->
      # Do things with response
      $.ajax(
        url: 'http://www.httpbin.org/headers'
        type: 'GET'
        success: (response) ->
          # Do things with the response
          callback(response)
      )
  )
  
console.log("Making requests...\n")

make_requests((response) ->
  console.log(response)
              
  $.ajax(
    url: 'http://www.httpbin.org/ip'
    type: 'GET'
    success: (response) ->
      console.log(response)
  )
)

## Pitfalls: Error Handling
Error handling can be tricky with callbacks. There are two approaches to handling errors raised in asynchronous function. 

### `error` argument
One way is to make the callback take an extra argument representing the error. The callback is then responsible for checking for the existence of this argument and acting accordingly. 

### Error callback

The second approach is for the asynchronous function to take an error handling callback in addition to the normal one (like `$.ajax`)

In [18]:
%%coffee
make_request = (callback) ->
  # $.ajax takes two functions: success and error
  $.ajax(
    url: 'http://www.httpbin.org/status/404'
    type: 'GET'
    success: (response) ->
      # All is well, call the callback normally
      callback(response)
    error: (response) ->
      # Error: call the callback with an error argument
      callback(null, true)
  )
  
make_request((response, error) ->
  if error
    console.log("Something bad happened!")
    return  # Careful you don't forget the return!
    
  console.log("Request completed successfully")
)

Something bad happened!


When nesting and error handling combine, you can end up with hard-to-maintain code that can have subtle bugs. Here's the nesting example from before, this time with error handling:

In [20]:
%%coffee
make_requests = (callback) ->
  $.ajax(
    url: 'http://www.httpbin.org/status/404'
    type: 'GET'
    success: (response) ->
      # Do things with response
      $.ajax(
        url: 'http://www.httpbin.org/post'
        type: 'POST'
        success: (response) ->
          # Do things with the response
          callback(response)
        error: ->
          callback(null, true)
      )
    error: ->
      callback(null, true)
  )
  
console.log("Making requests...\n")
make_requests((response, error) ->
  if error
    console.log("Failed!")
    return
              
  console.log("Finished requests, making one more...\n")
    
  $.ajax(
    url: 'http://www.httpbin.org/delay/1'
    type: 'GET'
    success: (response) ->
      console.log("Done!")
    error: ->
      console.log("Failed! (Duplicate)")
  )
)

Making requests...

Failed!


There can be tons of duplicated code here, especially if you just want to a generic error handler for every failure case. There's no such thing as a `catch` block with callbacks, so you have to make sure you put error handling code at every level of nesting! 

## Downsides: Storing Asynchronous Results
Asynchronous functions are often used to fetch data that need to be used many times. The problem with callbacks is that there is no way to "return" data from them: they are called somewhere else and the return value is ignored.

In [21]:
%%coffee
# Attempt to "return" from callback
get_headers = (callback) ->
  $.ajax(
    url: 'http://www.httpbin.org/headers'
    type: 'GET'
    success: (response) ->
      callback(response)  # Drops the return value on the floor
  )
  return

headers = get_headers((headers) ->
  return headers
)
  
# Let's see what's in <headers>...
console.log("Headers: " + headers)

Headers: undefined


If we want to fetch provider data for rendering a page, how do we avoid making multiple expensive ajax requests? The best we can do is try to save the result into a variable and check it later. Let's try that.

In [22]:
%%coffee
cached_data = null

get_and_cache_data = (callback) ->
  console.log("Requesting data!")
  $.ajax(
    url: 'http://www.httpbin.org/delay/1'
    type: 'GET'
    success: ->
      cached_data = {data: "stuff"}
      
      if callback
        callback(cached_data)
  )
  return
  
get_and_cache_data()
  
# Somewhere else, we try to access the data

if cached_data
  console.log("Accessing cached data!")
  # Do things with cached_data
else
  get_and_cache_data((data) ->
    # Do same things with data. Duplicate code!
  )

Requesting data!
Requesting data!


We still made multiple ajax requests! What's worse, we have duplicate code trying to manipulate the data: once for if it's synchronous (because it's cached), and once if it's asychronous (because it's not cached yet and we have to fetch it.)

Although callbacks might seem fine for dead-simple use cases, they don't scale well to more complex problems.

# Promises

## What is a Promise?
A "promise" is an object that represents a value. We don't actually care whether or not the value is available now, we just want to be able to use it.

Let's try it out.



In [23]:
promise = $.Deferred()

promise.then(function(value) {
    console.log("Count: " + value)
})

{}

In [24]:
promise.resolve(23)

Count: 23


{}

## What just happened?
A promise has three states:

* **Pending** - The value is not available yet
* **Fulfilled** - The promise succeeded and the value is available
* **Rejected** - The promise failed for some reason

We say a promise has ***settled*** if it is fulfilled or rejected. We can fulfill or reject a promise by using [`.resolve()`](https://api.jquery.com/deferred.resolve/) and [`.reject()`](https://api.jquery.com/deferred.reject/), each of which take optional data to resolve/reject with.


## Getting the goods

There are three ways of accessing the value:

* [`.then(onFulfilled, onRejected)`](https://api.jquery.com/deferred.then/) - runs the appropriate function depending on whether the promise was fulfilled or rejected.
* [`.done(fn)`](https://api.jquery.com/deferred.done/) - runs the function if the promise was fulfilled.
* [`.fail(fn)`](https://api.jquery.com/deferred.fail/) - runs the function if the promise was rejected.

## Wait, this looks like callbacks again!
Well, not quite. The game changer is *chaining*. Let's look at `.then()`:

In [27]:
promise = $.Deferred()

promise.then(function(value) {
    return 2 + value
}).then(function(value) {
    console.log(value)
})

{}

In [28]:
promise.resolve(23)

25


{}

# What?!
Yeah. `.then()` returns a new promise that resolves to the return value of the function provided! 

`.done()` and `.fail()` return promises too, but these resolve to the same promise they were chained onto.

In [31]:
%%coffee
promise = $.Deferred().resolve(10)

promise.done((value) -> 
  return 0
)
promise.done((value) ->
    console.log(value)      
  )

10


Let's go through some of the callback pitfalls and see how promises cleans things up. 

## Nesting
No more nesting! Promises mean that we can chain where we would previously nest. Here's the "pyramid of doom" example from before, this time using promises.

In [32]:
%%coffee
make_requests = () ->
  promise = $.get('http://www.httpbin.org/delay/1')
    .then((response) -> 
      return $.getJSON('http://www.httpbin.org/headers')
    )
  return promise
  
console.log("Making requests...\n")

make_requests()
  .then((response) ->
    console.log(response)
    return $.getJSON('http://www.httpbin.org/ip')
  )
  .then((response) -> 
    console.log(response)
  )

Making requests...

{"headers":{"Accept":"*/*, application/json, text/javascript, */*; q=0.01","Host":"www.httpbin.org","User-Agent":"node-XMLHttpRequest"}}
{"origin":"209.117.82.227"}


## Error Handling
If a promise fails, then everything chained onto it also fails. This lets us use `.fail()` like a `catch` block and avoid duplicating code!

In [45]:
%%coffee
promise = $.get('http://www.httpbin.org/status/404')

promise.then((response) ->
  return $.get('http://www.httpbin.org/headers')
  )
  .then((response) ->
    return $.get('http://www.httpbin.org/delay/1')     
  , (response) ->
    console.log("Failed!")
    return $.Deferred().resolve({data: "success"})
  )
  .then((response) ->
    console.log(response)
  )
  .fail(->
    console.log("Operation failed!")     
  )

Failed!
{"data":"success"}


None of the `.then()`s executed! This makes sense: if a line throws an exception in a `try` block, we don't continue executing the next line; we jump straight to the `catch`. With promises, all the error handlers get called.
## Storing results
It doesn't matter whether the promise has settled or not, the interface is the same. If you chain onto a settled promise, the function executes immediately (and synchronously.) This means we get caching out of the box!

In [35]:
// Return data after one second
promise = $.get('http://www.httpbin.org/delay/1').then(function(){
    return {data: "stuff"}
})
    
promise.then(function(data) {
    console.log(data)
})

{}

{"data":"stuff"}


In [36]:
promise.then(function(data) {
    console.log(data)
})

{"data":"stuff"}


{}

The second access to `data_promise` executes immediately because it's already been fulfilled. Once a promise has settled, it's locked in and cannot be changed. This is similar to callbacks: once the callback is called, the value can't change mid-function!

This lets us pass around promise objects like they're normal variables. And since the only way to access that data is through the chaining functions, there's no duplicated code for synchronous vs. asynchronous accesses.

# Using promises in your code
Here's just a couple of useful things you can use promises for in your code.

## Faking a backend
Sometimes you will find yourself needing a backend that doesn't exist yet. Promises let you abstract away the fetch so that the caller's code doesn't change when the backend is implemented.

In [37]:
%%coffee
# Supposed to fetch a number from the server
fetch_random_number = ->
  return $.Deferred().resolve(4)

# Somewhere else...
fetch_random_number().then((value) ->
  # No idea if an ajax fetch actually happened, I just want the value
  console.log("Completely random dice roll: " + value)
)

Completely random dice roll: 4


## Waiting for multiple ajax requests
Don't even ask how this would be done with callbacks. Use `$.when()`! 

`$.when` takes promises as its arguments and returns a mega-promise that resolves when all the promises resolve. The mega-promise resolves to a series of values, one for each promise passed into the `when()`. Each one is an array representing the arguments that the promise resolved to. If even one of the promises is rejected, the mega-promise is immediately rejected as well.

In [39]:
%%coffee
headers = $.getJSON('http://www.httpbin.org/headers')
ip = $.getJSON('http://www.httpbin.org/ip')

$.when(headers, ip)
  .then((headers_response, ip_response) ->
    # Don't run this until everything succeeded!
    console.log(headers_response)
    console.log('')
    console.log(ip_response)
  )
  .fail(->
    console.log("oh no!")     
  )

[Array] [{"headers":{"Accept":"*/*, application/json, text/javascript, */*; q=0.01","Host":"www.httpbin.org","User-Agent":"node-XMLHttpRequest"}},"success",{"readyState":4,"responseText":"{\n  \"headers\": {\n    \"Accept\": \"*/*, application/json, text/javascript, */*; q=0.01\", \n    \"Host\": \"www.httpbin.org\", \n    \"User-Agent\": \"node-XMLHttpRequest\"\n  }\n}\n","responseJSON":{"headers":{"Accept":"*/*, application/json, text/javascript, */*; q=0.01","Host":"www.httpbin.org","User-Agent":"node-XMLHttpRequest"}},"status":200,"statusText":"success"}]

[Array] [{"origin":"209.117.82.227"},"success",{"readyState":4,"responseText":"{\n  \"origin\": \"209.117.82.227\"\n}\n","responseJSON":{"origin":"209.117.82.227"},"status":200,"statusText":"success"}]


## Converting from Callbacks
Sometimes we are forced to use a callback-driven API. Examples of this include `setTimeout` and jQuery's `.on()` for registering event listeners. If you have functions that make such a call, you don't have to take a callback! Instead, create a promise and resolve it where you would otherwise call the callback.

In [40]:
%%coffee
hide_modal = ->
  promise = $.Deferred()
  
  # Pretend this is $('#modal').on('hide', fn)
  setTimeout(->
    promise.resolve()
  , 1000)
  
  return promise

console.log("Hiding modal...\n")
hide_modal().then(->
  console.log("Modal hidden!")                 
)

Hiding modal...

Modal hidden!


# Differences with Promise/A+ spec

`$.Deferred` is not compliant with the Promises spec. The most important difference to know is that it doesn't do error handling properly. First, let's look at ECMA6 Promises.

In [41]:
%%coffee
promise = new Promise((resolve, reject) ->
  resolve()
)
promise.then(->
  throw Error("Oh no!")
)
  .catch((reason) ->  # .catch() == .fail()
    console.log(reason.message)      
  )

Oh no!


And now let's look at how `Deferred` objects handle the same situation.

In [42]:
%%coffee
promise = $.Deferred()
promise.then(->
  throw Error("Oh no!")            
)
  .fail((reason) ->
    console.log(reason)
  )
promise.resolve()

Javascript Error: Oh no!

The error is completely uncaught and crashes the entire application! The way around this is to return a rejected promise instead of throwing an error. It's not a perfect solution, but it's still better than callbacks.

In [43]:
%%coffee
promise = $.Deferred()
promise.then(->
  return $.Deferred().reject("Oh no!")            
)
  .fail((reason) ->
    console.log(reason)
  )
promise.resolve()

Oh no!


In [ ]:
%%coffee
promise = $.Deferred()

promise.promise()